In [1]:
from keras.callbacks import ModelCheckpoint
from keras.metrics import categorical_accuracy
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend
import tensorflow as tf
import numpy as np
import json

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
sess = tf.Session(config=config) 
backend.set_session(sess)

In [3]:
class LeNet:
  @staticmethod
  def build(n_channels, 
            rows, 
            cols, 
            n_classes, 
            activation='relu', 
            weights_path=None):
    model = Sequential()
    
    input_Shape = (rows, cols, n_channels)
    
   
    #Convolution and Pooling Layers
    model.add(Conv2D(20, kernel_size=5, activation=activation, input_shape=input_Shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(50, kernel_size=5, activation=activation))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    #Flatten and Fully Conected layer
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation(activation))
    
    # OutputLayer SOFTMAX activation
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    
    # if a weights path is supplied (inicating that the model was
    # pre-trained), then load the weights
    if weights_path is not None:
      model.load_weights(weights_path)
    
    return model

In [4]:
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

In [5]:
n_epochs = 10
ini_learning_rate = 1e-3
batch_size = 32

In [6]:
with tf.Session() as sess:
    train_model = LeNet.build(n_channels=1, 
                              rows=28,
                              cols=28, 
                              n_classes=26, 
                              activation='relu', 
                              weights_path=None)

    opt = Adam(lr=ini_learning_rate, decay=ini_learning_rate / n_epochs)
    train_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[categorical_accuracy])
    try:
        file_path = 'victor_weights_Letras.h5'
        check_point = ModelCheckpoint(file_path, 
                                      monitor='categorical_accuracy', 
                                      verbose=0, 
                                      save_best_only=True,
                                      mode='max',
                                      save_weights_only=True)
        callbacks_list = [check_point]
        
        train_history = train_model.fit(x=x_train, 
                                        y=y_train, 
                                        batch_size=batch_size, 
                                        epochs=n_epochs,
                                        callbacks=callbacks_list,
                                        verbose=1)
    
   
    except IOError:
        print('Error while saving the Model weights')

sess.close()

Epoch 1/10
14649/14649 [==============================] - 4s 248us/step - loss: 0.6909 - categorical_accuracy: 0.8057
Epoch 2/10
14649/14649 [==============================] - 2s 138us/step - loss: 0.1907 - categorical_accuracy: 0.9470
Epoch 3/10
14649/14649 [==============================] - 2s 138us/step - loss: 0.1455 - categorical_accuracy: 0.9590
Epoch 4/10
14649/14649 [==============================] - 2s 138us/step - loss: 0.1144 - categorical_accuracy: 0.9690
Epoch 5/10
14649/14649 [==============================] - 2s 139us/step - loss: 0.1054 - categorical_accuracy: 0.9721
Epoch 6/10
14649/14649 [==============================] - 2s 138us/step - loss: 0.0918 - categorical_accuracy: 0.9771
Epoch 7/10
14649/14649 [==============================] - 2s 145us/step - loss: 0.0889 - categorical_accuracy: 0.9771
Epoch 8/10
14649/14649 [==============================] - 2s 147us/step - loss: 0.0681 - categorical_accuracy: 0.9814
Epoch 9/10
14649/14649 [==============================] 

In [7]:
json.dump(train_history.history, open('history/victor', 'w'))